In [2]:
import pandas as pd
import numpy as np

# Número de registros a generar
num_records = 2000

# Creación del diccionario de datos
data = {
    # === Variables Demográficas ===
    'EdadJefeHogar': np.random.randint(18, 80, size=num_records),
    'SexoJefeHogar': np.random.choice(['Hombre', 'Mujer'], size=num_records, p=[0.7, 0.3]),
    'MiembrosHogar': np.random.randint(1, 10, size=num_records),
    'RatioDependencia': np.random.uniform(0, 3, size=num_records).round(2), # (Niños + Ancianos) / Adultos
    'LenguaMaterna': np.random.choice(['Español', 'Quechua', 'Aymara', 'Otra'], size=num_records, p=[0.85, 0.12, 0.02, 0.01]),

    # === Variables Socioeconómicas ===
    'NivelEducativoJefeHogar': np.random.choice(['Ninguno', 'Primaria', 'Secundaria', 'Superior'], size=num_records, p=[0.1, 0.3, 0.4, 0.2]),
    'AniosEstudioJefeHogar': np.random.randint(0, 22, size=num_records),
    'TipoEmpleo': np.random.choice(['Formal', 'Informal', 'Desempleado'], size=num_records, p=[0.3, 0.6, 0.1]),
    'IngresoMensualHogar': np.random.lognormal(mean=7.5, sigma=0.8, size=num_records).round(2) * 3, # Ingreso en Soles
    'GastoMensualHogar': lambda: data['IngresoMensualHogar'] * np.random.uniform(0.6, 1.1, size=num_records),

    # === Variables de Vivienda y Activos ===
    'AreaResidencia': np.random.choice(['Urbana', 'Rural'], size=num_records, p=[0.75, 0.25]),
    'TenenciaVivienda': np.random.choice(['Propia', 'Alquilada', 'Cedida'], size=num_records, p=[0.6, 0.3, 0.1]),
    'MaterialParedes': np.random.choice(['Ladrillo/Cemento', 'Adobe', 'Madera/Esteras'], size=num_records, p=[0.65, 0.25, 0.1]),
    'AccesoAguaPotable': np.random.choice([1, 0], size=num_records, p=[0.8, 0.2]), # Dicotómica (1: Sí, 0: No)
    'AccesoSaneamiento': np.random.choice([1, 0], size=num_records, p=[0.7, 0.3]), # Dicotómica (1: Sí, 0: No)
    'AccesoElectricidad': np.random.choice([1, 0], size=num_records, p=[0.9, 0.1]), # Dicotómica (1: Sí, 0: No)
    'Hacinamiento': np.random.choice([1, 0], size=num_records, p=[0.3, 0.7]), # Dicotómica (1: Sí, 0: No)
    'PoseeActivos': np.random.choice([1, 0], size=num_records, p=[0.6, 0.4]), # Dicotómica (1: Sí, 0: No)
    
    # === Variable Geográfica ===
    'Region': np.random.choice(['Costa', 'Sierra', 'Selva'], size=num_records, p=[0.5, 0.3, 0.2])
}

# Creación del DataFrame
df = pd.DataFrame(data)

# Ajuste del gasto para que no sea una función anónima
df['GastoMensualHogar'] = (df['IngresoMensualHogar'] * np.random.uniform(0.7, 1.2, size=num_records)).round(2)

# === Creación de la Variable Objetivo: PobrezaMonetaria ===
# La probabilidad de ser pobre aumenta con:
# - Bajo nivel educativo y pocos años de estudio
# - Empleo informal o desempleo
# - Residencia en área rural o en la Sierra/Selva
# - Malas condiciones de vivienda y falta de servicios básicos
# - Mayor número de miembros en el hogar

probabilidad_pobreza = 0.1

probabilidad_pobreza += (df['NivelEducativoJefeHogar'] == 'Ninguno') * 0.25
probabilidad_pobreza += (df['NivelEducativoJefeHogar'] == 'Primaria') * 0.15
probabilidad_pobreza += (df['TipoEmpleo'] == 'Informal') * 0.15
probabilidad_pobreza += (df['TipoEmpleo'] == 'Desempleado') * 0.25
probabilidad_pobreza += (df['AreaResidencia'] == 'Rural') * 0.20
probabilidad_pobreza += (df['Region'].isin(['Sierra', 'Selva'])) * 0.10
probabilidad_pobreza += (df['AccesoAguaPotable'] == 0) * 0.10
probabilidad_pobreza += (df['AccesoSaneamiento'] == 0) * 0.10
probabilidad_pobreza += (df['MiembrosHogar'] > 4) * 0.10
probabilidad_pobreza += (df['Hacinamiento'] == 1) * 0.08
probabilidad_pobreza -= (df['IngresoMensualHogar'] > 2000) * 0.20 # Línea de pobreza aproximada
probabilidad_pobreza -= (df['PoseeActivos'] == 1) * 0.05

# Asegurar que la probabilidad esté entre 0 y 1
probabilidad_pobreza = np.clip(probabilidad_pobreza, 0.05, 0.95)

df['PobrezaMonetaria'] = (np.random.rand(num_records) < probabilidad_pobreza).astype(int)

# Reordenar columnas para tener la variable objetivo al final
columnas = [col for col in df.columns if col != 'PobrezaMonetaria'] + ['PobrezaMonetaria']
df = df[columnas]

# Guardar en archivo CSV
df.to_csv('prediccion_pobreza_peru.csv', index=False)

print("Base de datos 'prediccion_pobreza_peru.csv' creada exitosamente.")
print(df.head())
print("\nDescripción de la base de datos:")
print(df.info())

Base de datos 'prediccion_pobreza_peru.csv' creada exitosamente.
   EdadJefeHogar SexoJefeHogar  MiembrosHogar  RatioDependencia LenguaMaterna  \
0             22        Hombre              9              2.53       Español   
1             55        Hombre              4              1.60       Español   
2             70        Hombre              6              2.51       Español   
3             62        Hombre              8              2.28       Quechua   
4             42        Hombre              3              0.65       Español   

  NivelEducativoJefeHogar  AniosEstudioJefeHogar TipoEmpleo  \
0              Secundaria                     19     Formal   
1                Superior                      4   Informal   
2                 Ninguno                      8     Formal   
3              Secundaria                     10   Informal   
4                Superior                     18   Informal   

   IngresoMensualHogar  GastoMensualHogar AreaResidencia TenenciaVivi